In [1]:
import pandas as pd
import numpy as np
from os import walk, makedirs
import matplotlib.pyplot as plt
from copy import deepcopy
from statistics import mean, stdev

##### read data in

In [2]:
results_dir = '../results/fewshot'
fewshot_names =['bert-base-multilingual-cased-sentiment-multilingual',
                'dehatebert-mono-french',
                'french_xlm_xnli',
                'xlm-roberta-base-sentiment-multilingual']
result_dict = {}

for name in fewshot_names:
  result_dict.update({name:pd.read_csv(f"{results_dir}/{name}").sample(frac=1)})

result_dict

{'bert-base-multilingual-cased-sentiment-multilingual':                                                 model       traindata  \
 20  bert-base-multilingual-cased-sentiment-multili...   1epoch_train8   
 11  bert-base-multilingual-cased-sentiment-multili...   3epoch_train4   
 23  bert-base-multilingual-cased-sentiment-multili...   3epoch_train2   
 16  bert-base-multilingual-cased-sentiment-multili...   2epoch_train4   
 27  bert-base-multilingual-cased-sentiment-multili...   2epoch_train8   
 8   bert-base-multilingual-cased-sentiment-multili...   1epoch_train2   
 19  bert-base-multilingual-cased-sentiment-multili...  4epoch_train22   
 28  bert-base-multilingual-cased-sentiment-multili...   4epoch_train8   
 18  bert-base-multilingual-cased-sentiment-multili...   5epoch_train8   
 5   bert-base-multilingual-cased-sentiment-multili...  4epoch_train16   
 7   bert-base-multilingual-cased-sentiment-multili...  5epoch_train16   
 2   bert-base-multilingual-cased-sentiment-multili...  1

##### make line graph for train size

In [ ]:
fig, ax = plt.subplots(layout='constrained')
# plt.xticks(rotation=90)

for key, value in output_dict.items():
  rects = ax.bar(x + offset, b, width, label=key)
  # rects = ax.bar(x + offset, list(value.get('a_mean')), width, label=key)
  # ax.bar_label(rects, padding=3)
  multiplier += 1

ax.set_ylabel('Time in ms')
ax.set_title('Problem2_B_values')

no_thread = deepcopy(colnames)
no_thread.remove("thread")

ax.set_xticks(x + width, no_thread)
ax.legend(keys)

plt.show()